# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
from pathlib import Path
import psycopg2
import pandas as pd
from sql_queries import create_sql_insert_query
from typing import List
import json

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
os.getcwd()

'c:\\Users\\tew\\Training\\data_engineering_udacity\\3_Data_Modeling_With_Postgres'

In [5]:
def find_all_files_in_subdirectories(dir_path: os.PathLike, extension: str) -> List:
    list_files_paths = [
        path_to_file
        for path_to_file in Path(dir_path).rglob(f'*.{extension}')
    ]
    return list_files_paths
    

In [6]:
songs_directory_path = os.path.join(r"data","song_data")
song_files = find_all_files_in_subdirectories(songs_directory_path, "json")


In [7]:

filepath = song_files[0]

In [8]:
with open(filepath) as f:
    file_dict = json.load(f)
    data = { col:[val] for col,val in file_dict.items() }
    df = pd.DataFrame.from_dict(data)
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARD7TVE1187B99BFB1,None,None,California - LA,Casual,SOMZWCG12A8C13C480,I Didn't Mean To,218.93179,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [9]:
df.columns

Index(['num_songs', 'artist_id', 'artist_latitude', 'artist_longitude',
       'artist_location', 'artist_name', 'song_id', 'title', 'duration',
       'year'],
      dtype='object')

In [10]:
cols = ['song_id', 'title', 'artist_id', 'year', 'duration']
song_data = df[cols].values
song_data

array([['SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1',
        0, 218.93179]], dtype=object)

In [11]:
df[cols].values.tolist()

[['SOMZWCG12A8C13C480',
  "I Didn't Mean To",
  'ARD7TVE1187B99BFB1',
  0,
  218.93179]]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [12]:
column_to_value_data = dict(zip(cols,df[cols].values.tolist()[0]))
song_insert_query, song_values = create_sql_insert_query("songs", column_to_value_data=column_to_value_data)

In [13]:
song_insert_query

'\n        INSERT INTO songs\n        (song_id, title, artist_id, year, duration)\n        VALUES (%s, %s, %s, %s, %s);\n    '

In [14]:

cur.execute(song_insert_query, song_values)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [15]:
cols = ['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']

df[cols]

,artist_id,artist_name,artist_location,artist_latitude,artist_longitude
0,ARD7TVE1187B99BFB1,Casual,California - LA,None,None


In [16]:
artist_data = df[cols].values
artist_data

array([['ARD7TVE1187B99BFB1', 'Casual', 'California - LA', None, None]],
      dtype=object)

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [17]:
sql_artists_table_cols = ['artist_id', 'name', 'location', 'latitude', 'longitude']
column_to_value_data = dict(zip(sql_artists_table_cols,artist_data.tolist()[0]))
artist_insert_query, artist_values = create_sql_insert_query("artists", column_to_value_data=column_to_value_data)

In [18]:
artist_insert_query

'\n        INSERT INTO artists\n        (artist_id, name, location, latitude, longitude)\n        VALUES (%s, %s, %s, %s, %s);\n    '

In [19]:
cur.execute(artist_insert_query, artist_values)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [22]:
log_directory_path = os.path.join(r"data","log_data")
log_directory_path

'data\\log_data'

In [24]:
log_files =  find_all_files_in_subdirectories(log_directory_path, "json")
len(log_files)

30

In [31]:
filepath = log_files[0]
filepath


WindowsPath('data/log_data/2018/11/2018-11-01-events.json')

In [36]:
file_dict, data

({'num_songs': 1,
  'artist_id': 'ARD7TVE1187B99BFB1',
  'artist_latitude': None,
  'artist_longitude': None,
  'artist_location': 'California - LA',
  'artist_name': 'Casual',
  'song_id': 'SOMZWCG12A8C13C480',
  'title': "I Didn't Mean To",
  'duration': 218.93179,
  'year': 0},
 {'num_songs': [1],
  'artist_id': ['ARD7TVE1187B99BFB1'],
  'artist_latitude': [None],
  'artist_longitude': [None],
  'artist_location': ['California - LA'],
  'artist_name': ['Casual'],
  'song_id': ['SOMZWCG12A8C13C480'],
  'title': ["I Didn't Mean To"],
  'duration': [218.93179],
  'year': [0]})

In [39]:
data = []
with open(filepath) as f:
    for line in f:
        data.append(json.loads(line))
len(data)

15

In [40]:
df = pd.DataFrame(data)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Walter,M,0,Frye,NaN,free,"San Francisco-Oakland-Hayward, CA",GET,Home,1.540919e+12,38,None,200,1541105830796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",39
1,None,Logged In,Kaylee,F,0,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Home,1.540345e+12,139,None,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
3,None,Logged In,Kaylee,F,2,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Upgrade,1.540345e+12,139,None,200,1541106132796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [ ]:
df = 
df.head()

In [ ]:
t = 
t.head()

In [ ]:
time_data = ()
column_labels = ()

In [ ]:
time_df = 
time_df.head()

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [ ]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [ ]:
user_df = 

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [ ]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [ ]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = ()
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.